In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras import layers
import os
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read the video and extract info about it
cap = cv2.VideoCapture('/content/drive/MyDrive/test.mp4')


In [ ]:
# get total number of frames and generate a list with each 30 th frame 
totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
x = [i for i in range (1, totalFrames) if divmod(i, int(30))[1]==0]
print(totalFrames)

369


In [ ]:
currentframe=0
for myFrameNumber in x:
  #set which frame to read
  cap.set(cv2.CAP_PROP_POS_FRAMES,myFrameNumber)
  # read frame
  ret, frame = cap.read()
  # display frame
  name = '/content/drive/MyDrive/test_set/test_frame_' + str(currentframe) + '.jpg'
  print ('Creating...' + name)
  cv2.imwrite(name, frame)
  currentframe += 1

  # wait one second, exit loop if escape is pressed
  ch = 0xFF & cv2.waitKey(1000)
  if ch == 27:
    break

Creating.../content/drive/MyDrive/test_set/test_frame_0.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_1.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_2.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_3.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_4.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_5.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_6.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_7.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_8.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_9.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_10.jpg
Creating.../content/drive/MyDrive/test_set/test_frame_11.jpg


In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)
model = tf.keras.Model(pre_trained_model.input, x)
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])
checkpoint_path = "/content/drive/MyDrive/training/cp.ckpt"
if os.path.exists('/content/drive/MyDrive/training'):
  model.load_weights(checkpoint_path)
else:
  print("Train your model")

In [ ]:
clas = ['Batting','Boundary']
for i in range(0,currentframe):
  img_path = '/content/drive/MyDrive/test_set/test_frame_' + str(i) + '.jpg'
  img = tf.keras.utils.load_img(
      img_path, target_size=(224, 224)
  )
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  print(score)
  print(
      "This image most likely belongs to {} with a {:.2f} percent confidence."
      .format(clas[np.argmax(score)], 100 * np.max(score))
  )

tf.Tensor([0.73041034 0.2695896 ], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.04 percent confidence.
tf.Tensor([0.7310586  0.26894143], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.11 percent confidence.
tf.Tensor([0.7310586  0.26894143], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.11 percent confidence.
tf.Tensor([0.7310586  0.26894143], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.11 percent confidence.
tf.Tensor([0.7310579 0.2689421], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.11 percent confidence.
tf.Tensor([0.7310586  0.26894143], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.11 percent confidence.
tf.Tensor([0.7310586  0.26894143], shape=(2,), dtype=float32)
This image most likely belongs to Batting with a 73.11 percent confidence.
tf.Tensor([0.7310586  0.26894143], shape=(2